In [145]:
from pymongo import MongoClient
import csv
import pandas as pd
import urllib.request
import json
from flask import send_from_directory
import os
from pymongo import MongoClient
from flask import Flask, render_template
import urllib.request
from utils import *


In [146]:
client = MongoClient('localhost', 27017)  # or MongoClient("localhost:27")
db = client.bigdata_hw
collection = db.authors_final


In [147]:
##import csv into pandas
df = pd.read_csv('books_clean.csv')
#drop null columns
df = df.dropna(how="all")
df.rename(columns={'  num_pages': 'num_pages'}, inplace=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 11123 entries, 0 to 11126
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              11123 non-null  float64
 1   title               11123 non-null  object 
 2   authors             11123 non-null  object 
 3   average_rating      11123 non-null  float64
 4   isbn                11123 non-null  object 
 5   isbn13              11123 non-null  float64
 6   language_code       11123 non-null  object 
 7   num_pages           11123 non-null  float64
 8   ratings_count       11123 non-null  float64
 9   text_reviews_count  11123 non-null  float64
 10  publication_date    11123 non-null  object 
 11  publisher           11123 non-null  object 
dtypes: float64(6), object(6)
memory usage: 1.1+ MB


In [148]:
#split the authors string into a list with different authors and then explode into different rows
df=df.assign(authors=df.authors.str.split('/')).explode('authors')

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19237 entries, 0 to 11126
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              19237 non-null  float64
 1   title               19237 non-null  object 
 2   authors             19237 non-null  object 
 3   average_rating      19237 non-null  float64
 4   isbn                19237 non-null  object 
 5   isbn13              19237 non-null  float64
 6   language_code       19237 non-null  object 
 7   num_pages           19237 non-null  float64
 8   ratings_count       19237 non-null  float64
 9   text_reviews_count  19237 non-null  float64
 10  publication_date    19237 non-null  object 
 11  publisher           19237 non-null  object 
dtypes: float64(6), object(6)
memory usage: 1.9+ MB


In [150]:
import json
auth_data=df.groupby('authors').apply(lambda x: x.to_dict(orient='records'))
auth_data
df_grouped = pd.DataFrame(auth_data)
df_grouped
dff=df_grouped.rename(columns={0:'books'}).reset_index()
dff
df_dict = dff.to_dict('records')
df_dict


[{'authors': 'A. Bartlett Giamatti',
  'books': [{'bookID': 31253.0,
    'title': 'Baseball: a Literary Anthology',
    'authors': 'A. Bartlett Giamatti',
    'average_rating': 4.24,
    'isbn': '193108209X',
    'isbn13': 9780000000000.0,
    'language_code': 'eng',
    'num_pages': 733.0,
    'ratings_count': 182.0,
    'text_reviews_count': 14.0,
    'publication_date': '3/4/2002',
    'publisher': 'Library of America'}]},
 {'authors': 'A. Elizabeth Delany',
  'books': [{'bookID': 17118.0,
    'title': "Having Our Say: The Delany Sisters' First 100 Years",
    'authors': 'A. Elizabeth Delany',
    'average_rating': 4.02,
    'isbn': '822215020',
    'isbn13': 9780000000000.0,
    'language_code': 'eng',
    'num_pages': 62.0,
    'ratings_count': 81.0,
    'text_reviews_count': 9.0,
    'publication_date': '3/1/1996',
    'publisher': 'Dramatists Play Service'}]},
 {'authors': 'A. Merritt',
  'books': [{'bookID': 23596.0,
    'title': 'Tales Before Tolkien: The Roots of Modern Fanta

In [151]:
collection.insert_many(df_dict)


In [152]:
#DO NOT RUN THIS UNLESS YOU WAN TO DROP DATA
# collection.drop()

## mongodb insert complete of initial data

## Find all the books by JK Rowling

In [153]:
rowling_books = collection.find({"authors": "J.K. Rowling"})

count = 0
for bk in rowling_books:
    # for book in books:
    print(bk['books'])
    for book in bk['books']:
        print(book['title'])
        count += 1

print(f'\n\n no of books by JK Rowling: {count}')


[{'bookID': 1.0, 'title': 'Harry Potter and the Half-Blood Prince (Harry Potter  #6)', 'authors': 'J.K. Rowling', 'average_rating': 4.57, 'isbn': '439785960', 'isbn13': 9780000000000.0, 'language_code': 'eng', 'num_pages': 652.0, 'ratings_count': 2095690.0, 'text_reviews_count': 27591.0, 'publication_date': '9/16/2006', 'publisher': 'Scholastic Inc.'}, {'bookID': 2.0, 'title': 'Harry Potter and the Order of the Phoenix (Harry Potter  #5)', 'authors': 'J.K. Rowling', 'average_rating': 4.49, 'isbn': '439358078', 'isbn13': 9780000000000.0, 'language_code': 'eng', 'num_pages': 870.0, 'ratings_count': 2153167.0, 'text_reviews_count': 29221.0, 'publication_date': '9/1/2004', 'publisher': 'Scholastic Inc.'}, {'bookID': 4.0, 'title': 'Harry Potter and the Chamber of Secrets (Harry Potter  #2)', 'authors': 'J.K. Rowling', 'average_rating': 4.42, 'isbn': '439554896', 'isbn13': 9780000000000.0, 'language_code': 'eng', 'num_pages': 352.0, 'ratings_count': 6333.0, 'text_reviews_count': 244.0, 'publ

## Find all the books by Stephen King

In [154]:
King_books = collection.find({"authors": "Stephen King"})

count = 0
for bk in King_books:
    # for book in books:
    print(bk['books'])
    for book in bk['books']:
        print(book['title'])
        count += 1


print(f'\n\nno of books by Stephen King:{count}')


[{'bookID': 4978.0, 'title': 'Wolves of the Calla (The Dark Tower  #5)', 'authors': 'Stephen King', 'average_rating': 4.18, 'isbn': '141651693X', 'isbn13': 9780000000000.0, 'language_code': 'eng', 'num_pages': 931.0, 'ratings_count': 127168.0, 'text_reviews_count': 2801.0, 'publication_date': '2/1/2006', 'publisher': 'Pocket Books'}, {'bookID': 5094.0, 'title': 'The Drawing of the Three (The Dark Tower  #2)', 'authors': 'Stephen King', 'average_rating': 4.23, 'isbn': '451210859', 'isbn13': 9780000000000.0, 'language_code': 'eng', 'num_pages': 463.0, 'ratings_count': 173022.0, 'text_reviews_count': 5139.0, 'publication_date': '8/5/2003', 'publisher': 'Signet'}, {'bookID': 5373.0, 'title': 'The Waste Lands (The Dark Tower  #3)', 'authors': 'Stephen King', 'average_rating': 4.24, 'isbn': '747411875', 'isbn13': 9780000000000.0, 'language_code': 'eng', 'num_pages': 509.0, 'ratings_count': 76.0, 'text_reviews_count': 5.0, 'publication_date': '11/12/1992', 'publisher': 'Time Warner Paperbacks

## Find all books by Gabriel García Márquez

In [155]:
marq_books = collection.find({"authors": "Gabriel García Márquez"})

count = 0
for bk in marq_books:
    # for book in books:
    print(bk['books'])
    for book in bk['books']:
        print(book['title'])
        count += 1

print(f'\n\nno of books by Gabriel García Márquez : {count}')


[{'bookID': 324.0, 'title': 'Cien años de soledad', 'authors': 'Gabriel García Márquez', 'average_rating': 4.07, 'isbn': '785950109', 'isbn13': 9780000000000.0, 'language_code': 'spa', 'num_pages': 448.0, 'ratings_count': 63.0, 'text_reviews_count': 7.0, 'publication_date': '1/1/1990', 'publisher': 'French & European'}, {'bookID': 759.0, 'title': 'Collected Stories', 'authors': 'Gabriel García Márquez', 'average_rating': 4.19, 'isbn': '60932686', 'isbn13': 9780000000000.0, 'language_code': 'eng', 'num_pages': 352.0, 'ratings_count': 6375.0, 'text_reviews_count': 179.0, 'publication_date': '5/13/2008', 'publisher': 'Harper Perennial Modern Classics'}, {'bookID': 762.0, 'title': 'Crónica de una muerte anunciada', 'authors': 'Gabriel García Márquez', 'average_rating': 3.97, 'isbn': '1400034957', 'isbn13': 9780000000000.0, 'language_code': 'spa', 'num_pages': 118.0, 'ratings_count': 7888.0, 'text_reviews_count': 411.0, 'publication_date': '10/14/2003', 'publisher': 'Vintage Espanol'}, {'bo

## Find from API and insert auththor bio and works

Using each author name, conduct a search on https://openlibrary.org for the author’s JSON document:<br>
<ul><li>To find author (example: https://openlibrary.org/search/authors.json?q=firstname%20lastname</li>
<li>In the resulting doc(s), find your author’s key</li>
<li>Use that key (for example: OL123456) to find their works and bio info: https://openlibrary.org/authors/OL123456.json</li><ul>

## All API functions

In [179]:
from flask import Flask, render_template
import urllib.request
import json


# function to return the auth key
def return_auth_key(s):
    url = 'https://openlibrary.org/search/authors.json?q=' + \
        ('+'.join(s.split(' ')))

    response = urllib.request.urlopen(url)
    data = response.read()
    dict_auth = json.loads(data)
    auth_key = dict_auth['docs'][0]['key']
    return auth_key


# function to return the auth bio
def return_auth_bio(auth_key):
    url = f'https://openlibrary.org/authors/{auth_key}.json'
    response = urllib.request.urlopen(url)
    data = response.read()
    auth_bio = json.loads(data)
    return auth_bio

# function to return the works


def return_auth_works(auth_key):
    url = f'https://openlibrary.org/authors/{auth_key}/works.json'
    response = urllib.request.urlopen(url)
    data = response.read()
    auth_works = json.loads(data)
    return auth_works


#return the isbns of all the books : PS. THERE ARE SOME DUPLICATE TITLE SO THEY WILL BE DISPLAYED ONCE
def return_all_isbn(author_books):
    dict_isbn = {}
    for bk in author_books:
        for book in bk['books']:
            # print(book['isbn'])
            dict_isbn[book['title']] = '0' + \
                book['isbn'] if len(book['isbn']) < 10 else book['isbn']
    return dict_isbn


#this will return the amazonID for given isbn but then isbn can be used by amazon link directly so no use . This is only for the books in kaggle datadet as they already have isbn
def return_amazon_links(dict_isbn):
    amazon_id = {}

    for title, isbn in dict_isbn.items():
        url = f"https://openlibrary.org/api/books?bibkeys=ISBN:{isbn}&jscmd=data&format=json"

        try:
            response = urllib.request.urlopen(url)
            data = response.read()
            dict_book = json.loads(data)
            key1 = list(dict_book.keys())[0]
            iden = dict_book[key1]['identifiers']['amazon'][-1]
            # print(iden)
            if iden != '':
                amazon_id[title] = (iden)
        except:
            # print('no data')
            pass

    return amazon_id


#since the works dont have isbn--> we have to fetch some edition using the API for each work then we can use that edition to get amazon ID for the link.
def return_amazon_works(rowling_books):
    work_title_keys = {}
    for bk in rowling_books:
        works = (bk['works']['entries'])
        for work in works:
            work_title_keys[work['title']] = work['key']

    work_title_links = {}
    for title, work_id in work_title_keys.items():
        url = f'https://openlibrary.org{work_id}/editions.json'
        response = urllib.request.urlopen(url)
        data = response.read()
        dict_editions_info = json.loads(data)
        editions = dict_editions_info['entries']

        print(title)
        for edition in editions:
            try:
                for link in edition['source_records']:
                    if(link[:7] == 'amazon:'):
                        print(link[7:])
                        work_title_links[title] = link[7:]
                        break

            except:
                print('no data')

            else:
                break

    return work_title_links


def return_all_works(auth_info):
    all_works = []
    for auth in auth_info:
        works = auth['works']['entries']
        for work in works:
            all_works.append(work['title'])

    return all_works


### JK Rowling

In [157]:
auth_key=return_auth_key('J. K. Rowling')
bio=return_auth_bio(auth_key)
works=return_auth_works(auth_key)
auth_key



'OL23919A'

In [158]:
collection.update_one({"authors": "J.K. Rowling"}, {"$set": {"bio": bio}})
collection.update_one({"authors": "J.K. Rowling"}, {"$set": {"works": works}})


### Stephen King

In [159]:
auth_key = return_auth_key('Stephen King')
bio=return_auth_bio(auth_key)
works=return_auth_works(auth_key)


In [160]:
collection.update_one({"authors": "Stephen King"}, {"$set": {"bio": bio}})
collection.update_one({"authors": "Stephen King"}, {"$set": {"works": works}})


### Gabriel Garcia Marquez

In [161]:
auth_key=return_auth_key('Gabriel Garcia Marquez')
bio=return_auth_bio(auth_key)
works=return_auth_works(auth_key)


In [162]:
collection.update_one({"authors": "Gabriel García Márquez"}, {"$set": {"bio": bio}})
collection.update_one({"authors": "Gabriel García Márquez"}, {
                      "$set": {"works": works}})


## Store the amazon links for only the works from API call

In [163]:
#here the links to the amazonIDs for the works of the 3 authors are stored to be used to render them in the flask app. 

jk_amazon_works = return_amazon_works(collection.find({"authors": "J.K. Rowling"}))
sk_amazon_works=return_amazon_works(collection.find({"authors": "Stephen King"}))
gg_amazon_works = return_amazon_works(
    collection.find({"authors": "Gabriel García Márquez"}))

author_keys_works = {
    return_auth_key('J. K. Rowling'): jk_amazon_works,
    return_auth_key('Stephen King'): sk_amazon_works,
    return_auth_key('Gabriel Garcia Marquez'): gg_amazon_works
}


Harry Potter, Tome 5, Harry Potter et l'ordre du Phénix
no data
บังอู๋ครีเอท
no data
no data
Harry Potter and the Half-Blood Prince [2/2]
4915512576
Harry Potter and the Goblet of Fire [1/4]
Harry Potter és a tűz serlege
no data
Harry Potter a tajemná komnata
Harry Potter Hardcover Boxed Set
1338864289
Harry Potter and the Half-Blood Prince
Harry Potter e a Pedra Filosofal
9722365541
Harry Potter and the Chamber of Secrets
060619181X
Harry Potter and the Prisoner of Azkaban
0563492627
Harry Potter and the Half-Blood Prince (SparkNotes Literature Guide)
Harry Potter and the Chamber of Secrets (SparkNotes Literature Guide)
Harry Potter and the Deathly Hallows
no data
Harry Potter
Harry Potter and the Prisoner of Azkaban (SparkNotes Literature Guide)
Harry Potter and the Order of the Phoenix (SparkNotes Literature Guide)
Harry Potter and the Goblet of Fire
no data
9750848365
Harry Potter and the Prisoner of Azkaban Transforming Pictures Book
Harry Potter e il prigioniero di Azkaban
886256

### Flask app

In [182]:
from flask import send_from_directory
import os
from pymongo import MongoClient
from flask import Flask, render_template
import urllib.request
# import json
from utils import *

app = Flask(__name__)

author_list = [
    'J. K. Rowling',
    'Stephen King',
    'Gabriel Garcia Marquez'
]
auth_key_dict = {auth: return_auth_key(auth) for auth in author_list}


client = MongoClient('localhost', 27017)
db = client.bigdata_hw
collection = db.authors_final

#home route
@app.route('/')
def index():
    return render_template('index.html', author_key_dict=auth_key_dict)

#avoid extra call for the tab icon
@app.route('/favicon.ico')
def favicon():
    return send_from_directory(os.path.join(app.root_path, 'static'),
                               'favicon.ico', mimetype='image/vnd.microsoft.icon')

#dynamic route for the author
@app.route('/<id>')
def author_page(id):
    print(id)
    auth_key = return_auth_key(id)
    print(auth_key)
    query = {"bio.key": f'/authors/{auth_key}'}
    
    author = collection.find(query)
    return render_template('author.html', id_author=collection.find(query), akey=auth_key, books_amazon=return_all_isbn(author), works_amazon=author_keys_works[auth_key])


if __name__ == '__main__':
    app.run(host='localhost', port=5001)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5001
Press CTRL+C to quit
127.0.0.1 - - [15/Mar/2023 23:53:43] "GET / HTTP/1.1" 200 -


J. K. Rowling


127.0.0.1 - - [15/Mar/2023 23:53:45] "GET /J.%20K.%20Rowling HTTP/1.1" 200 -


OL23919A
Stephen King


127.0.0.1 - - [15/Mar/2023 23:53:55] "GET /Stephen%20King HTTP/1.1" 200 -


OL2162284A
Gabriel Garcia Marquez


127.0.0.1 - - [15/Mar/2023 23:54:00] "GET /Gabriel%20Garcia%20Marquez HTTP/1.1" 200 -


OL4586796A


In [167]:
def return_all_works(auth_info):
    all_works=[]
    for auth in auth_info:
        works=auth['works']['entries']
        for work in works:
            all_works.append(work['title'])
    
    
    return all_works
    
    

In [180]:
return_all_works(collection.find({"authors": "J.K. Rowling"}))


["Harry Potter, Tome 5, Harry Potter et l'ordre du Phénix",
 'บังอู๋ครีเอท',
 'Harry Potter and the Half-Blood Prince [2/2]',
 'Harry Potter and the Goblet of Fire [1/4]',
 'Harry Potter és a tűz serlege',
 'Harry Potter a tajemná komnata',
 'Harry Potter Hardcover Boxed Set',
 'Harry Potter and the Half-Blood Prince',
 'Harry Potter e a Pedra Filosofal',
 'Harry Potter and the Chamber of Secrets',
 'Harry Potter and the Prisoner of Azkaban',
 'Harry Potter and the Half-Blood Prince (SparkNotes Literature Guide)',
 'Harry Potter and the Chamber of Secrets (SparkNotes Literature Guide)',
 'Harry Potter and the Deathly Hallows',
 'Harry Potter',
 'Harry Potter and the Prisoner of Azkaban (SparkNotes Literature Guide)',
 'Harry Potter and the Order of the Phoenix (SparkNotes Literature Guide)',
 'Harry Potter and the Goblet of Fire',
 'Harry Potter and the Prisoner of Azkaban Transforming Pictures Book',
 'Harry Potter e il prigioniero di Azkaban',
 "Harry Potter and the Philosopher's Sto